# Import libraries

In [1]:
!pip install bertopic  umap-learn sentence_transformers pyvi hdbscan gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 77.4 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 47.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 72.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install underthesea

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 75.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 42.1 MB/s eta 0:00:00


In [3]:
from transformers import AutoModel, AutoTokenizer
from bertopic import BERTopic
from bertopic.backend import BaseEmbedder
import torch
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from pyvi.ViTokenizer import tokenize
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.representation import KeyBERTInspired
import itertools
import numpy as np
import json
import os
from collections import defaultdict
import pandas as pd
from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora.dictionary import Dictionary
from underthesea import word_tokenize
from torch.utils.data import Dataset

2025-06-14 12:54:16.069399: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749905656.243059      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749905656.293902      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Data

In [4]:
def convert_to_dataframe(data):
    rows = []
    for event_id, articles in enumerate(data):
        for article in articles:
            rows.append({
                "event_id": event_id + 1,
                "text": article
            })
    df = pd.DataFrame(rows)
    df["id"] = range(len(df))
    return df[["id", "event_id", "text"]]

In [5]:
# Nhập link data all_events_dataset.txt (cần được chia train/val/test cho giống id bên Uyên)
with open(r"/kaggle/input/vims-data-zyzy/all_events_dataset.txt", "r", encoding="utf-8") as f:
    data = json.load(f)
df = convert_to_dataframe(data)
print(df)

        id  event_id                                               text
0        0         1  Các nước chia buồn vụ tai nạn máy bay của Ai C...
1        1         1  Máy bay Ai Cập rơi: Những câu hỏi cho chính ph...
2        2         1  Giải mã bí ẩn máy bay rơi của EgyptAir . Phó C...
3        3         1  Phát hiện mảnh vỡ nghi của máy bay MS804 gặp n...
4        4         1  Máy bay EgyptAir có thể bị tấn công bằng tên l...
...    ...       ...                                                ...
1940  1940       300  Xử lý xong sự cố ở sân bay Buôn Ma Thuột . Chi...
1941  1941       300  Đã khắc phục xong sự cố đường băng sân bay Buô...
1942  1942       300  Khắc phục xong sự cố Cảng hàng không Buôn Ma T...
1943  1943       300  Khắc phục xong sự cố tại Cảng Hàng không Buôn ...
1944  1944       300  Sân bay Buôn Mê Thuột hoạt động trở lại . Trướ...

[1945 rows x 3 columns]


In [6]:
# Nhập link train/val/test của Uyên vào uyn_data
def convert_uyn_dataset_to_dataframe(uyn_data):
    with open(uyn_data, 'r', encoding='utf-8') as f:
        data = json.load(f)

    df = pd.DataFrame(data)[['id']]
    return df

In [7]:
train = convert_uyn_dataset_to_dataframe(r"/kaggle/input/vims-data-zyzy/train.json")
train = pd.merge(train, df, on='id', how='inner')
print(len(train))
print(train)

1575
        id  event_id                                               text
0     1094       168  Emilia Clarke: "Mỗi cảnh khỏa thân trong "Game...
1     1387       211  Hoàng Anh Gia Lai liên tục bị thu hồi dự án . ...
2      404        61  ‘Mặc Irina Shayk nằm nhà chờ, Ronaldo chỉ muốn...
3     1380       210  EU cảnh báo cá chết bất thường, thủy sản có kh...
4     1674       257  Hoàng Thuỳ Linh đã sẵn sàng công khai yêu Vĩnh...
...    ...       ...                                                ...
1570  1565       240  Dòi bò nhung nhúc trong các ụ chứa mắm ở Vĩnh ...
1571  1899       293  Rơi từ tầng 3 công trình xây dựng, 1 người tử ...
1572  1532       235  Cháy siêu thị điện máy 3 tầng ở Quảng Trị . Kh...
1573  1369       209  Vingroup công bố chiết khấu 0% cho doanh nghiệ...
1574  1025       157  Nghi can tấn công làm vợ chết, chồng bị thương...

[1575 rows x 3 columns]


In [8]:
test = convert_uyn_dataset_to_dataframe(r"/kaggle/input/vims-data-zyzy/test.json")
test = pd.merge(test, df, on='id', how='inner')
print(len(test))
print(test)

195
       id  event_id                                               text
0    1112       171  Đông Nhi và Ông Cao Thắng tìm kiếm nhóm nhạc n...
1    1929       298  Máy bay Nga chở 147 hành khách bất ngờ... tụt ...
2     640        99  Nhà báo thông tin ‘2 mặt’ không ổn . Tại họp b...
3    1379       210  Cảnh báo về tình trạng thủy sản nhiễm hóa chất...
4      28         4  Cần chặn dòng người ồ ạt di cư vào TP.HCM . TS...
..    ...       ...                                                ...
190   593        91  Hoàng Xuân Vinh đoạt HCĐ World Cup . Tại vòng ...
191   921       143  Không thu lệ phí phúc khảo bài thi THPT quốc g...
192  1803       277  Ưu đãi lớn cho một số mẫu xe máy Honda . Theo ...
193  1490       228  Công Phượng có thể làm khán giả ở Myanmar vì c...
194  1435       219  “Du khách nhí” đổ về Suối Tiên dự Lễ hội trái ...

[195 rows x 3 columns]


In [9]:
validation = convert_uyn_dataset_to_dataframe(r"/kaggle/input/vims-data-zyzy/validation.json")
validation = pd.merge(validation, df, on='id', how='inner')
print(len(validation))
print(validation)

175
       id  event_id                                               text
0     615        95  Hối lộ CSGT nửa tỷ đồng để bỏ qua hơn nửa tấn ...
1    1266       194  EVN xin lỗi khách hàng vì sự cố mất điện tại 2...
2      90        13  Phát động giải báo chí “Búa liềm vàng” . Mỗi n...
3     636        98  Hai chuyên cơ dự phòng phục vụ Tổng thống Obam...
4     206        32  Thủ tướng dự Lễ khởi công Tổ hợp nuôi bò, chế ...
..    ...       ...                                                ...
170  1589       244  Từ chối lời cầu hôn, cô gái 18 tuổi bị thiêu s...
171  1523       234  Xe khách bất ngờ nổ tung, 8 người Việt thiệt m...
172  1890       292  143 tác phẩm vào vòng chung khảo Giải Báo chí ...
173   359        54  Chàng trai “khùng” đạp xe xuyên Việt vận động ...
174  1204       185  Cái kết không như mơ của chàng trai trốn lên m...

[175 rows x 3 columns]


In [10]:
# Fine tune không dùng val nên merge train+val = new train
train = pd.concat([train, validation], ignore_index=True)
print(len(train))
print(train)

1750
        id  event_id                                               text
0     1094       168  Emilia Clarke: "Mỗi cảnh khỏa thân trong "Game...
1     1387       211  Hoàng Anh Gia Lai liên tục bị thu hồi dự án . ...
2      404        61  ‘Mặc Irina Shayk nằm nhà chờ, Ronaldo chỉ muốn...
3     1380       210  EU cảnh báo cá chết bất thường, thủy sản có kh...
4     1674       257  Hoàng Thuỳ Linh đã sẵn sàng công khai yêu Vĩnh...
...    ...       ...                                                ...
1745  1589       244  Từ chối lời cầu hôn, cô gái 18 tuổi bị thiêu s...
1746  1523       234  Xe khách bất ngờ nổ tung, 8 người Việt thiệt m...
1747  1890       292  143 tác phẩm vào vòng chung khảo Giải Báo chí ...
1748   359        54  Chàng trai “khùng” đạp xe xuyên Việt vận động ...
1749  1204       185  Cái kết không như mơ của chàng trai trốn lên m...

[1750 rows x 3 columns]


In [11]:
train_list_of_docs = train['text'].tolist()
test_list_of_docs = test['text'].tolist()

# Embedding

In [12]:
class ContrastivePairDataset(Dataset):
    def __init__(self, pairs, tokenizer, max_length=256):
        self.pairs = pairs
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        text1, text2, label = self.pairs[idx]

        inputs1 = self.tokenizer(
            text1,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )
        inputs2 = self.tokenizer(
            text2,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )

        item = {
            'input_ids_1': inputs1['input_ids'].squeeze(0),
            'attention_mask_1': inputs1['attention_mask'].squeeze(0),
            'input_ids_2': inputs2['input_ids'].squeeze(0),
            'attention_mask_2': inputs2['attention_mask'].squeeze(0),
            'label': torch.tensor(label, dtype=torch.float)
        }
        return item

In [13]:
def make_contrastive_pairs(df, max_neg_per_pos=1):
    from collections import defaultdict
    import random

    grouped = defaultdict(list)
    for _, row in df.iterrows():
        grouped[row['event_id']].append(row['text'])

    pairs = []
    event_ids = list(grouped.keys())
    for eid in event_ids:
        texts = grouped[eid]
        for i in range(len(texts)):
            for j in range(i + 1, len(texts)):
                pairs.append((texts[i], texts[j], 1.0))  # positive pair

                for _ in range(max_neg_per_pos):
                    neg_eid = random.choice([e for e in event_ids if e != eid])
                    neg_text = random.choice(grouped[neg_eid])
                    pairs.append((texts[i], neg_text, 0.0))  # negative pair

    random.shuffle(pairs)
    return pairs

In [14]:
def get_cls_embedding(model, input_ids, attention_mask):
    output = model(input_ids=input_ids, attention_mask=attention_mask)
    return output.last_hidden_state[:, 0]  # CLS token

In [15]:
from torch import nn
from torch.utils.data import DataLoader

def train_phobert_contrastive(model, tokenizer, pairs, device="cpu", epochs=10, batch_size=8, lr=2e-5):
    dataset = ContrastivePairDataset(pairs, tokenizer)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    loss_fn = nn.CosineEmbeddingLoss()

    model.train()
    model.to(device)

    for epoch in range(epochs):
        total_loss = 0
        for batch in tqdm(dataloader, desc=f"Epoch {epoch+1}"):
            input_ids_1 = batch['input_ids_1'].to(device)
            attention_mask_1 = batch['attention_mask_1'].to(device)
            input_ids_2 = batch['input_ids_2'].to(device)
            attention_mask_2 = batch['attention_mask_2'].to(device)
            labels = batch['label'].to(device) * 2 - 1  # convert 0/1 to -1/+1

            emb1 = get_cls_embedding(model, input_ids_1, attention_mask_1)
            emb2 = get_cls_embedding(model, input_ids_2, attention_mask_2)

            loss = loss_fn(emb1, emb2, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch {epoch+1} Loss: {total_loss:.4f}")

    return model

In [16]:
# Khởi tạo model/tokenizer
model_name = "vinai/phobert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = AutoModel.from_pretrained(model_name)

# Chuẩn bị data
pairs = make_contrastive_pairs(train)

# Huấn luyện
trained_model = train_phobert_contrastive(model, tokenizer, pairs, device)
trained_model.save_pretrained("phobert_event_embedding")
tokenizer.save_pretrained("phobert_event_embedding")

config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

...

Epoch 9: 100%|██████████| 1158/1158 [17:07<00:00,  1.13it/s]


Epoch 9 Loss: 33.8569


Epoch 10: 100%|██████████| 1158/1158 [17:06<00:00,  1.13it/s]


Epoch 10 Loss: 30.7847


('phobert_event_embedding/tokenizer_config.json',
 'phobert_event_embedding/special_tokens_map.json',
 'phobert_event_embedding/vocab.txt',
 'phobert_event_embedding/bpe.codes',
 'phobert_event_embedding/added_tokens.json')

In [17]:
class PhoBERTEmbedder:
    def __init__(self, model_name="vinai/phobert-base", batch_size=32, max_length=256): 
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
        self.model = AutoModel.from_pretrained(model_name).to(self.device)
        self.batch_size = batch_size
        self.max_length = max_length

    def __call__(self, documents, show_progress_bar=False):
        self.model.eval()
        all_embeddings = []
        iterator = range(0, len(documents), self.batch_size)
        if show_progress_bar:
            iterator = tqdm(iterator, desc="Embedding documents")

        for i in iterator:
            batch_docs = documents[i:i+self.batch_size]
            tokens = self.tokenizer(
                batch_docs,
                padding=True,
                truncation=True,
                max_length=self.max_length,
                return_tensors="pt"
            ).to(self.device)

            with torch.no_grad():
                outputs = self.model(**tokens)
                cls_embeddings = outputs.last_hidden_state[:, 0, :]

            all_embeddings.append(cls_embeddings.cpu())

        return torch.cat(all_embeddings).numpy()

In [18]:
phobert_embedder = PhoBERTEmbedder(model_name="phobert_event_embedding")
embeddings = phobert_embedder(df['text'].tolist(), show_progress_bar=True)

Embedding documents: 100%|██████████| 61/61 [00:39<00:00,  1.53it/s]


# UMAP + HDBSCAN

In [19]:
# Grid tham số
umap_n_neighbors = [2, 4, 7, 10]
umap_n_components = [5]
umap_min_dist = [0.0, 0.1] 
umap_metric = ["cosine"] #, "manhattan", "euclidean"

hdbscan_min_cluster_size = [2, 4, 6, 8]
hdbscan_cluster_selection_method = ['eom', 'leaf'] 
hdbscan_metric = ['euclidean']

In [20]:
# Tổ hợp tham số
param_grid = list(itertools.product(
    umap_n_neighbors,
    umap_n_components,
    umap_min_dist,
    umap_metric,
    hdbscan_min_cluster_size,
    hdbscan_cluster_selection_method,
    hdbscan_metric
))

# Vectorizer

In [21]:
# Đọc stopword list từ file
with open(r"/kaggle/input/vims-data-zyzy/vietnamese-stopwords.txt", "r", encoding="utf-8") as f:
    my_stopwords = [line.strip() for line in f if line.strip()]

In [22]:
vectorizer_model = CountVectorizer(stop_words=my_stopwords, ngram_range=(1, 2)) #max_df=0.9, 

# Representation

In [23]:
keybert_model = KeyBERTInspired()
representation_model = {
    "KeyBERT": keybert_model
}

# Training

In [24]:
# Tìm best_model
best_model = None
best_score = -np.inf
best_params = None

for i, (n_neighbors, n_components, min_dist, umap_metric, min_cluster_size, cluster_selection_method, hdbscan_metric) in enumerate(param_grid):
    print(f"\n🔍 Grid {i+1}/{len(param_grid)}: UMAP(n_neighbors={n_neighbors}, min_dist={min_dist}, metric={umap_metric}), "
        f"HDBSCAN(min_cluster_size={min_cluster_size}, cluster_selection_method={cluster_selection_method}, metric={hdbscan_metric})")

    # Tạo UMAP và HDBSCAN
    umap_model = UMAP(n_neighbors=n_neighbors, n_components=n_components, min_dist=min_dist, metric=umap_metric, random_state=42)
    hdbscan_model = HDBSCAN(min_cluster_size=min_cluster_size, cluster_selection_method=cluster_selection_method, metric=hdbscan_metric, prediction_data=True, gen_min_span_tree=True)

    # BERTopic
    topic_model = BERTopic(
        # embedding_model=lambda docs: phobert_embedder(docs, show_progress_bar=False),
        embedding_model=lambda docs: embeddings(docs, show_progress_bar=False),
        umap_model=umap_model,
        hdbscan_model=hdbscan_model,
        vectorizer_model=vectorizer_model,
        # ctfidf_model=ctfidf_model,
        representation_model=representation_model,
        language="multilingual",
        calculate_probabilities=True,
        verbose=False
    )

    # Train
    topics, _ = topic_model.fit_transform(train['text'].tolist())
    # topics, _ = topic_model.fit_transform(list_of_docs, embeddings)

    # Tính DBCV score
    dbcv_score = topic_model.hdbscan_model.relative_validity_

    # Tính % outliers
    outlier_ratio = topics.count(-1) / len(topics)

    print(f"📊 DBCV: {round(dbcv_score, 3)} | Outlier Ratio: {round(outlier_ratio * 100, 2)}%")

    # Kết hợp: Ưu tiên DBCV cao nhất, sau đó là outlier thấp
    composite_score = dbcv_score - outlier_ratio  # hoặc dùng trọng số: alpha*DBCV - beta*outlier

    if composite_score > best_score:
        best_score = composite_score
        best_model = topic_model
        best_params = (n_neighbors, n_components, min_dist, umap_metric, min_cluster_size, cluster_selection_method, hdbscan_metric)

print(best_params)

topics, probs = best_model.fit_transform(train['text'].tolist())
# topics, probs = best_model.fit_transform(list_of_docs, embeddings)


🔍 Grid 1/64: UMAP(n_neighbors=2, min_dist=0.0, metric=cosine), HDBSCAN(min_cluster_size=2, cluster_selection_method=eom, metric=euclidean)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

📊 DBCV: 0.734 | Outlier Ratio: 4.51%

🔍 Grid 2/64: UMAP(n_neighbors=2, min_dist=0.0, metric=cosine), HDBSCAN(min_cluster_size=2, cluster_selection_method=leaf, metric=euclidean)
📊 DBCV: 0.731 | Outlier Ratio: 4.51%

🔍 Grid 3/64: UMAP(n_neighbors=2, min_dist=0.0, metric=cosine), HDBSCAN(min_cluster_size=4, cluster_selection_method=eom, metric=euclidean)
📊 DBCV: 0.428 | Outlier Ratio: 16.4%

🔍 Grid 4/64: UMAP(n_neighbors=2, min_dist=0.0, metric=cosine), HDBSCAN(min_cluster_size=4, cluster_selection_method=leaf, metric=euclidean)
📊 DBCV: 0.395 | Outlier Ratio: 14.74%

🔍 Grid 5/64: UMAP(n_neighbors=2, min_dist=0.0, metric=cosine), HDBSCAN(min_cluster_size=6, cluster_selection_method=eom, metric=euclidean)
📊 DBCV: 0.25 | Outlier Ratio: 29.03%

🔍 Grid 6/64: UMAP(n_neighbors=2, min_dist=0.0, metric=cosine), HDBSCAN(min_cluster_size=6, cluster_selection_method=leaf, metric=euclidean)
📊 DBCV: 0.225 | Outlier Ratio: 31.03%

🔍 Grid 7/64: UMAP(n_neighbors=2, min_dist=0.0, metric=cosine), HDBSCAN(m

In [25]:
# Tính DBCV score
dbcv_score = best_model.hdbscan_model.relative_validity_

# Tính % outliers
outlier_ratio = topics.count(-1) / len(topics)

print(f"📊 DBCV: {round(dbcv_score, 3)} | Outlier Ratio: {round(outlier_ratio * 100, 2)}%")

📊 DBCV: 0.734 | Outlier Ratio: 4.06%


In [26]:
train['topic'] = topics
train

,id,event_id,text,topic
0,1094,168,"Emilia Clarke: ""Mỗi cảnh khỏa thân trong ""Game...",156
1,1387,211,Hoàng Anh Gia Lai liên tục bị thu hồi dự án . ...,28
2,404,61,"‘Mặc Irina Shayk nằm nhà chờ, Ronaldo chỉ muốn...",148
3,1380,210,"EU cảnh báo cá chết bất thường, thủy sản có kh...",-1
4,1674,257,Hoàng Thuỳ Linh đã sẵn sàng công khai yêu Vĩnh...,98
...,...,...,...,...
1745,1589,244,"Từ chối lời cầu hôn, cô gái 18 tuổi bị thiêu s...",66
1746,1523,234,"Xe khách bất ngờ nổ tung, 8 người Việt thiệt m...",322
1747,1890,292,143 tác phẩm vào vòng chung khảo Giải Báo chí ...,9
1748,359,54,Chàng trai “khùng” đạp xe xuyên Việt vận động ...,332


In [27]:
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
from sklearn.preprocessing import LabelEncoder

train_eval = train[train["topic"] != -1].copy()
train_ari = adjusted_rand_score(train_eval["event_id"], train_eval["topic"])
train_nmi = normalized_mutual_info_score(train_eval["event_id"], train_eval["topic"])

print(f"Train ARI: {train_ari:.4f}")
print(f"Train NMI: {train_nmi:.4f}")

Train ARI: 0.6217
Train NMI: 0.9261


In [28]:
topic_to_docs = defaultdict(list)

for doc, topic in zip(train_list_of_docs, topics):
    topic_to_docs[topic].append(doc)

df = pd.DataFrame([
    {"Topic": topic, "Docs": topic_docs, "Count": len(topic_docs)}
    for topic, topic_docs in topic_to_docs.items()
])

result = []
for _, row in df.iterrows():
    result.append({
        "topic": row["Topic"],
        "count": row["Count"],
        "docs": row["Docs"]
    })

In [29]:
# JSON
with open(fr"/kaggle/working/Train_Event_Cluster.json", "w", encoding="utf-8") as f:
    json.dump(result, f, ensure_ascii=False, indent=2)

# Testing

In [30]:
# Tìm best_model
best_model = None
best_score = -np.inf
best_params = None

for i, (n_neighbors, n_components, min_dist, umap_metric, min_cluster_size, cluster_selection_method, hdbscan_metric) in enumerate(param_grid):
    print(f"\n🔍 Grid {i+1}/{len(param_grid)}: UMAP(n_neighbors={n_neighbors}, min_dist={min_dist}, metric={umap_metric}), "
        f"HDBSCAN(min_cluster_size={min_cluster_size}, cluster_selection_method={cluster_selection_method}, metric={hdbscan_metric})")

    # Tạo UMAP và HDBSCAN
    umap_model = UMAP(n_neighbors=n_neighbors, n_components=n_components, min_dist=min_dist, metric=umap_metric, random_state=42)
    hdbscan_model = HDBSCAN(min_cluster_size=min_cluster_size, cluster_selection_method=cluster_selection_method, metric=hdbscan_metric, prediction_data=True, gen_min_span_tree=True)

    # BERTopic
    topic_model = BERTopic(
        # embedding_model=lambda docs: phobert_embedder(docs, show_progress_bar=False),
        embedding_model=lambda docs: embeddings(docs, show_progress_bar=False),
        umap_model=umap_model,
        hdbscan_model=hdbscan_model,
        vectorizer_model=vectorizer_model,
        # ctfidf_model=ctfidf_model,
        representation_model=representation_model,
        language="multilingual",
        calculate_probabilities=True,
        verbose=False
    )

    # Train
    topics, _ = topic_model.fit_transform(test['text'].tolist())

    # Tính DBCV score
    dbcv_score = topic_model.hdbscan_model.relative_validity_

    # Tính % outliers
    outlier_ratio = topics.count(-1) / len(topics)

    print(f"📊 DBCV: {round(dbcv_score, 3)} | Outlier Ratio: {round(outlier_ratio * 100, 2)}%")

    # Kết hợp: Ưu tiên DBCV cao nhất, sau đó là outlier thấp
    composite_score = dbcv_score - outlier_ratio  # hoặc dùng trọng số: alpha*DBCV - beta*outlier

    if composite_score > best_score:
        best_score = composite_score
        best_model = topic_model
        best_params = (n_neighbors, n_components, min_dist, umap_metric, min_cluster_size, cluster_selection_method, hdbscan_metric)

print(best_params)

topics, probs = best_model.fit_transform(test['text'].tolist())


🔍 Grid 1/64: UMAP(n_neighbors=2, min_dist=0.0, metric=cosine), HDBSCAN(min_cluster_size=2, cluster_selection_method=eom, metric=euclidean)
📊 DBCV: 0.675 | Outlier Ratio: 8.72%

🔍 Grid 2/64: UMAP(n_neighbors=2, min_dist=0.0, metric=cosine), HDBSCAN(min_cluster_size=2, cluster_selection_method=leaf, metric=euclidean)
📊 DBCV: 0.675 | Outlier Ratio: 8.72%

🔍 Grid 3/64: UMAP(n_neighbors=2, min_dist=0.0, metric=cosine), HDBSCAN(min_cluster_size=4, cluster_selection_method=eom, metric=euclidean)
📊 DBCV: 0.361 | Outlier Ratio: 20.0%

🔍 Grid 4/64: UMAP(n_neighbors=2, min_dist=0.0, metric=cosine), HDBSCAN(min_cluster_size=4, cluster_selection_method=leaf, metric=euclidean)
📊 DBCV: 0.361 | Outlier Ratio: 20.0%

🔍 Grid 5/64: UMAP(n_neighbors=2, min_dist=0.0, metric=cosine), HDBSCAN(min_cluster_size=6, cluster_selection_method=eom, metric=euclidean)
📊 DBCV: 0.245 | Outlier Ratio: 29.23%

🔍 Grid 6/64: UMAP(n_neighbors=2, min_dist=0.0, metric=cosine), HDBSCAN(min_cluster_size=6, cluster_selection_me

In [31]:
# Tính DBCV score
dbcv_score = best_model.hdbscan_model.relative_validity_

# Tính % outliers
outlier_ratio = topics.count(-1) / len(topics)

print(f"📊 DBCV: {round(dbcv_score, 3)} | Outlier Ratio: {round(outlier_ratio * 100, 2)}%")

📊 DBCV: 0.716 | Outlier Ratio: 0.0%


In [32]:
test['topic'] = topics
test

,id,event_id,text,topic
0,1112,171,Đông Nhi và Ông Cao Thắng tìm kiếm nhóm nhạc n...,0
1,1929,298,Máy bay Nga chở 147 hành khách bất ngờ... tụt ...,0
2,640,99,Nhà báo thông tin ‘2 mặt’ không ổn . Tại họp b...,0
3,1379,210,Cảnh báo về tình trạng thủy sản nhiễm hóa chất...,0
4,28,4,Cần chặn dòng người ồ ạt di cư vào TP.HCM . TS...,0
...,...,...,...,...
190,593,91,Hoàng Xuân Vinh đoạt HCĐ World Cup . Tại vòng ...,0
191,921,143,Không thu lệ phí phúc khảo bài thi THPT quốc g...,0
192,1803,277,Ưu đãi lớn cho một số mẫu xe máy Honda . Theo ...,0
193,1490,228,Công Phượng có thể làm khán giả ở Myanmar vì c...,0


In [ ]:
train_eval = train[train["topic"] != -1].copy()
train_ari = adjusted_rand_score(train_eval["event_id"], train_eval["topic"])
train_nmi = normalized_mutual_info_score(train_eval["event_id"], train_eval["topic"])

print(f"Train ARI: {train_ari:.4f}")
print(f"Train NMI: {train_nmi:.4f}")

ARI: 0.7968
NMI: 0.9597


In [34]:
topic_to_docs = defaultdict(list)

for doc, topic in zip(test_list_of_docs, topics):
    topic_to_docs[topic].append(doc)

df = pd.DataFrame([
    {"Topic": topic, "Docs": topic_docs, "Count": len(topic_docs)}
    for topic, topic_docs in topic_to_docs.items()
])

result = []
for _, row in df.iterrows():
    result.append({
        "topic": row["Topic"],
        "count": row["Count"],
        "docs": row["Docs"]
    })

In [35]:
# JSON
with open(fr"/kaggle/working/Test_Event_Cluster.json", "w", encoding="utf-8") as f:
    json.dump(result, f, ensure_ascii=False, indent=2)